In [8]:
import os
import gzip

# Paramètres
input_file = "en.openfoodfacts.org.products.csv.gz"  # Fichier CSV.gz source
output_dir = "output_files1_csv"  # Dossier pour les fichiers de sortie
output_file = os.path.join(output_dir, "part_1.csv")  # Fichier de sortie unique
max_lines = 10000  # Nombre de lignes maximum dans le fichier, y compris les en-têtes

# Créer le dossier de sortie s'il n'existe pas
os.makedirs(output_dir, exist_ok=True)

# Taille totale du fichier
total_size_bytes = os.path.getsize(input_file)
print("Taille totale du fichier (Go):", total_size_bytes / (1024 ** 3))

# Lire et écrire les données dans un fichier CSV unique
with gzip.open(input_file, "rt", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    # Lire les en-têtes
    header = infile.readline()
    outfile.write(header)

    # Initialisation
    line_count = 0

    for line in infile:
        # Écrire la ligne dans le fichier de sortie
        outfile.write(line)
        line_count += 1

        # Arrêter après avoir écrit le nombre maximum de lignes
        if line_count >= max_lines:
            break

print(f"Fichier unique généré avec {line_count} lignes dans : {output_file}")


Taille totale du fichier (Go): 0.875
Fichier unique généré avec 10000 lignes dans : output_files1_csv/part_1.csv


## Chargement et comprehension du fichier csv


In [9]:
import pandas as pd
import os
# Chemin du fichier CSV
file_path = 'output_files1_csv/part_1.csv'

# Vérifier si le fichier existe
if not os.path.exists(file_path):
    print(f"Le fichier {file_path} n'existe pas.")
    exit()

# Chemin du fichier texte contenant les colonnes principales
columns_file_path = 'important_columns.txt'

# Charger les colonnes principales à partir du fichier texte
try:
    with open(columns_file_path, 'r') as file:
        selected_columns = [line.strip() for line in file.readlines() if line.strip()]
except Exception as e:
    print(f"Erreur lors du chargement des colonnes depuis le fichier texte : {e}")
    exit()

# Charger le fichier CSV
try:
    df = pd.read_csv(file_path, delimiter='\t', low_memory=False)
except Exception as e:
    print(f"Erreur lors du chargement du fichier : {e}")
    exit()

# Étape 1 : Garder uniquement les colonnes nécessaires
if all(col in df.columns for col in selected_columns):
    df = df[selected_columns]
else:
    print("Les colonnes sélectionnées ne sont pas toutes présentes dans le fichier.")
    exit()

# Étape 2 : Supprimer les doublons
df = df.drop_duplicates()


# Sauvegarder le fichier nettoyé
cleaned_file_path = 'cleaned_file.csv'
df.to_csv(cleaned_file_path, index=False, encoding='utf-8')

# Aperçu des premières lignes
print(f"Fichier nettoyé sauvegardé dans : {cleaned_file_path}")
print(df.head())

Les colonnes sélectionnées ne sont pas toutes présentes dans le fichier.
Fichier nettoyé sauvegardé dans : cleaned_file.csv
   code                                                url     creator  \
0    54  http://world-en.openfoodfacts.org/product/0000...     kiliweb   
1    63  http://world-en.openfoodfacts.org/product/0000...     kiliweb   
2     1  http://world-en.openfoodfacts.org/product/0000...         inf   
3     2  http://world-en.openfoodfacts.org/product/0000...     kiliweb   
4     3  http://world-en.openfoodfacts.org/product/0000...  prepperapp   

    created_t      created_datetime  last_modified_t last_modified_datetime  \
0  1582569031  2020-02-24T18:30:31Z       1733085204   2024-12-01T20:33:24Z   
1  1673620307  2023-01-13T14:31:47Z       1732913331   2024-11-29T20:48:51Z   
2  1634745456  2021-10-20T15:57:36Z       1734203090   2024-12-14T19:04:50Z   
3  1722606455  2024-08-02T13:47:35Z       1734216390   2024-12-14T22:46:30Z   
4  1716818343  2024-05-27T13:59:03Z 

In [3]:
!pip3 install rdflib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 9.1 MB/s eta 0:00:00


In [1]:
import csv
import urllib.parse
from rdflib import Graph, Namespace, URIRef, Literal, RDF, RDFS
from rdflib.namespace import OWL, XSD, FOAF, DC

# Chemins des fichiers
csv_file_path = "cleaned_file.csv"  # Remplacez par votre fichier CSV
txt_file_path = "important_columns.txt"  # Fichier texte contenant les colonnes importantes
rdf_file_path = "output_ontology.owl"  # Fichier de sortie OWL

# Espaces de noms pour l'ontologie
SCHEMA = Namespace("http://schema.org/")
DBO = Namespace("http://dbpedia.org/ontology/")
EX = Namespace("http://produitsalimentaires.fr/")
FOAF = Namespace("http://xmlns.com/foaf/0.1/")
XSD = Namespace("http://www.w3.org/2001/XMLSchema#")



# Charger la liste des colonnes importantes depuis le fichier texte
try:
    with open(txt_file_path, "r") as file:
        important_columns = [line.strip() for line in file.readlines() if line.strip()]
except Exception as e:
    print(f"Erreur lors du chargement des colonnes : {e}")
    exit()

# Liaison des namespaces au graphe
g = Graph()
g.bind("schema", SCHEMA)
g.bind("dbo", DBO)
g.bind("foaf", FOAF)
g.bind("dc", DC)
g.bind("ex", EX)
g.bind("owl", OWL)

# Définition de l'ontologie
g.add((URIRef("http://produitsalimentaires.fr/foaf-ontology"), RDF.type, OWL.Ontology))
g.add((URIRef("http://produitsalimentaires.fr/foaf-ontology"), RDFS.comment, Literal(
    "Une ontologie"
)))

# Définir les classes principales et sous-classes
g.add((EX.Product, RDF.type, OWL.Class))
g.add((DBO.Product, RDF.type, OWL.Class))

g.add((EX.Brand, RDF.type, OWL.Class))
g.add((DBO.Brand, RDF.type, OWL.Class))

g.add((EX.Category, RDF.type, OWL.Class))
g.add((DBO.Category, RDF.type, OWL.Class))

g.add((EX.Nutrient, RDF.type, OWL.Class))
g.add((EX.Allergen, RDF.type, OWL.Class))
#g.add((EX.Origin, RDF.type, OWL.Class))
#g.add((EX.Addiction, RDF.type, OWL.Class))
g.add((EX.Country, RDF.type, OWL.Class))
g.add((DBO.Country, RDF.type, OWL.Class))

g.add((EX.Creator, RDF.type, OWL.Class))
g.add((EX.ManufacturingPlace, RDF.type, OWL.Class))
g.add((EX.Ingredient, RDF.type, OWL.Class))
g.add((EX.PurchasePlace, RDF.type, OWL.Class))
g.add((EX.City, RDF.type, OWL.Class))
#g.add((EX.Ecoscore, RDF.type, OWL.Class))

# Sous-classes potentielles (exemples)
#g.add((EX.EcoscoreScore, RDF.type, OWL.Class))
#g.add((EX.EcoscoreGrade, RDF.type, OWL.Class))

# Définir les propriétés et leur domaine et plage
g.add((EX.hasBrand, RDF.type, OWL.ObjectProperty))
g.add((EX.hasBrand, RDFS.domain, EX.Product))
g.add((EX.hasBrand, RDFS.range, EX.Brand))

g.add((EX.hasCategory, RDF.type, OWL.ObjectProperty))
g.add((EX.hasCategory, RDFS.domain, EX.Product))
g.add((EX.hasCategory, RDFS.range, EX.Category))

#g.add((EX.hasOrigin, RDF.type, OWL.ObjectProperty))
#g.add((EX.hasOrigin, RDFS.domain, EX.Product))
#g.add((EX.hasOrigin, RDFS.range, EX.Origin))

g.add((EX.containsAllergen, RDF.type, OWL.ObjectProperty))
g.add((EX.containsAllergen, RDFS.domain, EX.Product))
g.add((EX.containsAllergen, RDFS.range, EX.Allergen))

g.add((EX.hasNutrient, RDF.type, OWL.ObjectProperty))
g.add((EX.hasNutrient, RDFS.domain, EX.Product))
g.add((EX.hasNutrient, RDFS.range, EX.Nutrient))

#g.add((EX.hasAddiction, RDF.type, OWL.ObjectProperty))
#g.add((EX.hasAddiction, RDFS.domain, EX.Product))
#g.add((EX.hasAddiction, RDFS.range, EX.Addiction))

g.add((EX.availableInCountry, RDF.type, OWL.ObjectProperty))
g.add((EX.availableInCountry, RDFS.domain, EX.Product))
g.add((EX.availableInCountry, RDFS.range, DBO.Country))

# Nouvelles propriétés pour les entités supplémentaires
g.add((EX.hasCreator, RDF.type, OWL.ObjectProperty))
g.add((EX.hasCreator, RDFS.domain, EX.Product))
g.add((EX.hasCreator, RDFS.range, EX.Creator))

g.add((EX.hasManufacturingPlace, RDF.type, OWL.ObjectProperty))
g.add((EX.hasManufacturingPlace, RDFS.domain, EX.Product))
g.add((EX.hasManufacturingPlace, RDFS.range, EX.ManufacturingPlace))

g.add((EX.hasIngredient, RDF.type, OWL.ObjectProperty))
g.add((EX.hasIngredient, RDFS.domain, EX.Product))
g.add((EX.hasIngredient, RDFS.range, EX.Ingredient))



g.add((EX.hasIngredientsText, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasIngredientsText, RDFS.domain, EX.Product))
g.add((EX.hasIngredientsText, RDFS.range, XSD.string))

# Ajouter les propriétés pour quantity et serving_size
g.add((EX.hasQuantity, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasQuantity, RDFS.domain, EX.Product))
g.add((EX.hasQuantity, RDFS.range, XSD.string))

#g.add((EX.hasServingSize, RDF.type, OWL.DatatypeProperty))
#g.add((EX.hasServingSize, RDFS.domain, EX.Product))
#g.add((EX.hasServingSize, RDFS.range, XSD.string))


g.add((EX.hasPurchasePlace, RDF.type, OWL.ObjectProperty))
g.add((EX.hasPurchasePlace, RDFS.domain, EX.Product))
g.add((EX.hasPurchasePlace, RDFS.range, EX.PurchasePlace))

g.add((EX.hasCity, RDF.type, OWL.ObjectProperty))
g.add((EX.hasCity, RDFS.domain, EX.Product))
g.add((EX.hasCity, RDFS.range, EX.City))

"""g.add((EX.hasEcoscore, RDF.type, OWL.ObjectProperty))
g.add((EX.hasEcoscore, RDFS.domain, EX.Product))
g.add((EX.hasEcoscore, RDFS.range, EX.Ecoscore))

g.add((EX.hasEcoscoreScore, RDF.type, OWL.ObjectProperty))
g.add((EX.hasEcoscoreScore, RDFS.domain, EX.Ecoscore))
g.add((EX.hasEcoscoreScore, RDFS.range, EX.EcoscoreScore))

#g.add((EX.hasEcoscoreGrade, RDF.type, OWL.ObjectProperty))
g.add((EX.hasEcoscoreGrade, RDFS.domain, EX.Ecoscore))
g.add((EX.hasEcoscoreGrade, RDFS.range, EX.EcoscoreGrade))"""

# Fonction pour encoder les URI de manière sécurisée
def encode_uri(uri):
    return urllib.parse.quote(uri, safe=":/#?&=~")

# Lire le fichier CSV et ajouter des instances
try:
    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:

            if all(col in row for col in important_columns):
                product_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/product/{row['code']}"))

                # Ajouter le type de produit

                g.add((product_uri, RDF.type, DBO.Product))
                g.add((product_uri, SCHEMA.name, Literal(row["product_name"])))
                g.add((product_uri, SCHEMA.url, URIRef(row["url"])))



                # Ajouter la marque
                if row["brands"]:
                    brand_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/brand/{row['brands']}"))
                    g.add((product_uri, EX.hasBrand, brand_uri))
                    g.add((brand_uri, RDF.type, DBO.Brand))
                    g.add((brand_uri, FOAF.name, Literal(row["brands"], lang="en")))

                # Ajouter des catégories basées sur le champ 'categories'
                if row["categories"]:
                    categories = row["categories"].split(',')  # Séparer les catégories par des virgules
                    for category in categories:
                        category = category.strip()
                        if category:  # Vérifier que la catégorie n'est pas vide
                            category_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/category/{category}"))

                            g.add((product_uri, EX.hasCategory, category_uri))
                            g.add((category_uri, RDF.type, EX.Category))
                            g.add((category_uri, FOAF.name, Literal(category, lang="en")))

                # Ajouter des relations d'origine basées sur le champ 'origins'
                #if row["origins"]:
                    #origins = row["origins"].split(',')  # Séparer les valeurs par des virgules
                    #for origin in origins:
                       # origin = origin.strip()
                        #if origin:  # Vérifier que l'origine n'est pas vide
                            # Créer une instance de l'origine
                           # origin_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/product/origin/{origin}"))

                            #g.add((product_uri, EX.hasOrigin, origin_uri))
                            #g.add((origin_uri, RDF.type, EX.Origin))
                            #g.add((origin_uri, SCHEMA.name, Literal(origin)))

                if row["allergens"]:
                    allergens = row["allergens"].split(',')  # Séparer les valeurs par des virgules
                    for allergen in allergens:
                        allergen = allergen.strip()
                        if allergen:  # Vérifier que l'origine n'est pas vide
                            # Créer une instance de l'origine
                            allergen_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/allergens/{allergen}"))

                            g.add((product_uri, EX.containsAllergen, allergen_uri))
                            g.add((allergen_uri, RDF.type, EX.Allergen))
                            g.add((allergen_uri, FOAF.name, Literal(allergen, lang="en")))

                # Ajouter des addictions
                #if row["additives"]:
                    #addiction_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/product/addiction/{row['additives']}"))
                    #print(f"Adding addictives URI: {addiction_uri}")
                    #g.add((product_uri, EX.hasAddiction, addiction_uri))
                    #g.add((addiction_uri, RDF.type, EX.Addiction))
                    #g.add((addiction_uri, SCHEMA.name, Literal(row["additives"])))

                # Ajouter des pays de disponibilité
                #if row["countries"]:
                   # country_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/country/{row['countries']}"))
                   # g.add((product_uri, EX.availableInCountry, country_uri))
                    #g.add((country_uri, RDF.type, DBO.Country))
                    #g.add((country_uri, SCHEMA.name, Literal(row["countries"])))

                if row["countries"]:
                    countries = row["countries"].split(',')  # Séparer les catégories par des virgules
                    for category in countries:
                        category = category.strip()
                        if category:  # Vérifier que la catégorie n'est pas vide
                            country_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/countries/{category}"))
                            g.add((product_uri, EX.availableInCountry, country_uri))
                            g.add((category_uri, RDF.type, DBO.Country))
                            g.add((category_uri, FOAF.name, Literal(category, lang="en")))

                # Ajouter des nutriments
                nutrients = [
                    ("energy_100g", "Energy"),
                    ("fat_100g", "Fat"),
                    ("saturated-fat_100g", "Saturated Fat"),
                    ("carbohydrates_100g", "Carbohydrates"),
                    ("sugars_100g", "Sugars"),
                    ("fiber_100g", "Fiber"),
                    ("proteins_100g", "Proteins"),
                    ("salt_100g", "Salt"),
                    ("carbon-footprint_100g", "carbon footprint")
                ]
                for nutrient, label in nutrients:
                    if row[nutrient]:
                        nutrient_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/nutrient/{label}"))
                        g.add((product_uri, EX.hasNutrient, nutrient_uri))
                        g.add((nutrient_uri, RDF.type, EX.Nutrient))
                        g.add((nutrient_uri, SCHEMA.name, Literal(label)))
                        g.add((nutrient_uri, SCHEMA.value, Literal(row[nutrient], datatype=XSD.float)))

                # Ajouter des ingrédients
                if row["ingredients_text"]:
                    g.add((product_uri, EX.hasIngredientsText, Literal(row["ingredients_text"])))

                # Ajouter les créateurs
                if row["creator"]:
                    creator_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/creator/{row['creator']}"))
                    g.add((product_uri, EX.hasCreator, creator_uri))
                    g.add((creator_uri, RDF.type, EX.Creator))
                    g.add((creator_uri, FOAF.name, Literal(row["creator"], lang="en")))

                # Ajouter les lieux de fabrication
                if row["manufacturing_places"]:
                    manufacturing = row["manufacturing_places"].split(',')  # Séparer les catégories par des virgules
                    for manu in manufacturing:
                        cmanu = manu.strip()
                        if category:  # Vérifier que la catégorie n'est pas vide

                            #Ajouter des instances
                            manufacturing_place_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/manufacturing_place/{manu}"))

                            # Définir les relations et propriétés
                            g.add((product_uri, EX.hasManufacturingPlace, manufacturing_place_uri))
                            g.add((manufacturing_place_uri, RDF.type, EX.ManufacturingPlace))
                            g.add((manufacturing_place_uri, FOAF.name, Literal(manu, lang="en")))



                # Ajouter les lieux d'achat
                if row["purchase_places"]:
                    purchase_place_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/purchase_place/{row['purchase_places']}"))
                    g.add((product_uri, EX.hasPurchasePlace, purchase_place_uri))
                    g.add((purchase_place_uri, RDF.type, EX.PurchasePlace))
                    g.add((purchase_place_uri, SCHEMA.name, Literal(row["purchase_places"])))

                # Ajouter les villes
                if row["cities"]:
                    city_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/city/{row['cities']}"))
                    g.add((product_uri, EX.hasCity, city_uri))
                    g.add((city_uri, RDF.type, EX.City))
                    g.add((city_uri, FOAF.name, Literal(row["cities"], lang="en")))

                  # Ajouter la quantité (quantity)
                if row["quantity"]:
                    g.add((product_uri, EX.hasQuantity, Literal(row["quantity"])))

                # Ajouter la taille de portion (serving_size)
                #if row["serving_size"]:
                   # g.add((product_uri, EX.hasServingSize, Literal(row["serving_size"])))

                # Ajouter l'écorescore (score et grade)


except Exception as e:
    print(f"Erreur lors du chargement du fichier CSV : {e}")
    exit()

# Sauvegarder le graphe OWL
g.serialize(destination=rdf_file_path, format="xml")
print(f"Ontologie OWL générée et sauvegardée dans : {rdf_file_path}")


Ontologie OWL générée et sauvegardée dans : output_ontology.owl


In [2]:
from rdflib import Graph

# Charger le graphe RDF depuis le fichier OWL
file_path = "output_ontology.owl"
g = Graph()
try:
    g.parse(file_path, format="xml")  # OWL est souvent au format RDF/XML
    print(f"Ontologie OWL chargée depuis : {file_path}")
except Exception as e:
    print(f"Erreur lors du chargement de l'ontologie OWL : {e}")
    exit()
# Sauvegarder le graphe au format JSON-LD
jsonld_file_path = "output_ontology.jsonld"
g.serialize(destination=jsonld_file_path, format="json-ld", indent=4)
print(f"Graphe RDF converti et sauvegardé au format JSON-LD dans : {jsonld_file_path}")

Ontologie OWL chargée depuis : output_ontology.owl
Graphe RDF converti et sauvegardé au format JSON-LD dans : output_ontology.jsonld


In [3]:
from rdflib import Graph

# Chemins des fichiers
owl_file_path = "output_ontology.owl"  # Chemin vers votre fichier OWL
ttl_file_path = "output_ontology.ttl"  # Chemin du fichier Turtle de sortie

# Charger le fichier OWL
g = Graph()
try:
    g.parse(owl_file_path, format="xml")  # OWL est souvent au format RDF/XML
    print(f"Ontologie OWL chargée depuis : {owl_file_path}")
except Exception as e:
    print(f"Erreur lors du chargement de l'ontologie OWL : {e}")
    exit()

# Sauvegarder le graphe en Turtle
try:
    g.serialize(destination=ttl_file_path, format="turtle")
    print(f"Fichier Turtle RDF généré avec succès dans : {ttl_file_path}")
except Exception as e:
    print(f"Erreur lors de la sauvegarde du fichier Turtle : {e}")

Ontologie OWL chargée depuis : output_ontology.owl
Fichier Turtle RDF généré avec succès dans : output_ontology.ttl


In [4]:
from rdflib import Graph

# Chemin vers le fichier source (.owl)
input_file_path = "output_ontology.owl"  # Remplacez par le chemin de votre fichier .owl

# Chemin vers le fichier de sortie (.rdf)
output_file_path = "output_ontology.rdf"  # Fichier de sortie au format RDF/XML


# Charger le fichier OWL dans un graphe RDFLib
g = Graph()
try:
    g.parse(input_file_path, format="xml")  # OWL est souvent au format RDF/XML
    print(f"Ontologie OWL chargée depuis : {input_file_path}")
except Exception as e:
    print(f"Erreur lors du chargement de l'ontologie OWL : {e}")
    exit()

# Sauvegarder le graphe en rdf
try:
    g.serialize(destination=output_file_path, format="xml")
    print(f"Fichier RDF généré avec succès dans : {output_file_path}")
except Exception as e:
    print(f"Erreur lors de la sauvegarde du fichier RDF : {e}")


print(f"Conversion terminée. Le fichier RDF a été enregistré sous : {output_file_path}")

Ontologie OWL chargée depuis : output_ontology.owl
Fichier RDF généré avec succès dans : output_ontology.rdf
Conversion terminée. Le fichier RDF a été enregistré sous : output_ontology.rdf


## Enrichir les Données avec des Liens vers DBpedia et Wikidata

In [5]:
!pip install SPARQLWrapper


In [5]:
#%pip install SPARQLWrapper
import csv
import json
import urllib.parse
import requests
from rdflib import Graph, Namespace, URIRef, Literal, RDF, RDFS
from rdflib.namespace import OWL, XSD, FOAF, DC
from SPARQLWrapper import SPARQLWrapper, JSON
import logging
import os

# Chemins des fichiers
txt_file_path = "important_columns.txt"  # Fichier texte contenant les colonnes importantes
rdf_file_path = "output_ontology2.rdf"  # Fichier de sortie OWL

# Espaces de noms pour l'ontologie
SCHEMA = Namespace("http://schema.org/")
DBO = Namespace("http://dbpedia.org/ontology/")
EX = Namespace("http://produitsalimentaires.fr/")
FOAF = Namespace("http://xmlns.com/foaf/0.1/")
XSD = Namespace("http://www.w3.org/2001/XMLSchema#")

# Charger la liste des colonnes importantes depuis le fichier texte
try:
    with open(txt_file_path, "r") as file:
        important_columns = [line.strip() for line in file.readlines() if line.strip()]
except Exception as e:
    print(f"Erreur lors du chargement des colonnes : {e}")
    exit()

# Liaison des namespaces au graphe
g = Graph()
g.bind("schema", SCHEMA)
g.bind("dbo", DBO)
g.bind("foaf", FOAF)
g.bind("dc", DC)
g.bind("ex", EX)
g.bind("owl", OWL)



# Définition de l'ontologie
g.add((URIRef("http://produitsalimentaires.fr/foaf-ontology"), RDF.type, OWL.Ontology))
g.add((URIRef("http://produitsalimentaires.fr/foaf-ontology"), RDFS.comment, Literal("Une ontologie")))

# Définir les classes principales et sous-classes
g.add((EX.Product, RDF.type, OWL.Class))
g.add((DBO.Product, RDF.type, OWL.Class))

g.add((EX.Brand, RDF.type, OWL.Class))
g.add((DBO.Brand, RDF.type, OWL.Class))

g.add((EX.Category, RDF.type, OWL.Class))
g.add((DBO.Category, RDF.type, OWL.Class))

g.add((EX.Nutrient, RDF.type, OWL.Class))
g.add((EX.Allergen, RDF.type, OWL.Class))

g.add((EX.Creator, RDF.type, OWL.Class))
g.add((EX.ManufacturingPlace, RDF.type, OWL.Class))
g.add((EX.Ingredient, RDF.type, OWL.Class))
g.add((EX.PurchasePlace, RDF.type, OWL.Class))
g.add((EX.City, RDF.type, OWL.Class))


# Définir les propriétés et leur domaine et plage
g.add((EX.hasBrand, RDF.type, OWL.ObjectProperty))
g.add((EX.hasBrand, RDFS.domain, EX.Product))
g.add((EX.hasBrand, RDFS.range, EX.Brand))
g.add((EX.Country, RDF.type, OWL.Class))
g.add((DBO.Country, RDF.type, OWL.Class))

g.add((EX.hasCategory, RDF.type, OWL.ObjectProperty))
g.add((EX.hasCategory, RDFS.domain, EX.Product))
g.add((EX.hasCategory, RDFS.range, EX.Category))


g.add((EX.containsAllergen, RDF.type, OWL.ObjectProperty))
g.add((EX.containsAllergen, RDFS.domain, EX.Product))
g.add((EX.containsAllergen, RDFS.range, EX.Allergen))

g.add((EX.hasNutrient, RDF.type, OWL.ObjectProperty))
g.add((EX.hasNutrient, RDFS.domain, EX.Product))
g.add((EX.hasNutrient, RDFS.range, EX.Nutrient))


g.add((EX.availableInCountry, RDF.type, OWL.ObjectProperty))
g.add((EX.availableInCountry, RDFS.domain, EX.Product))
g.add((EX.availableInCountry, RDFS.range, DBO.Country))

# Nouvelles propriétés pour les entités supplémentaires
g.add((EX.hasCreator, RDF.type, OWL.ObjectProperty))
g.add((EX.hasCreator, RDFS.domain, EX.Product))
g.add((EX.hasCreator, RDFS.range, EX.Creator))

g.add((EX.hasManufacturingPlace, RDF.type, OWL.ObjectProperty))
g.add((EX.hasManufacturingPlace, RDFS.domain, EX.Product))
g.add((EX.hasManufacturingPlace, RDFS.range, EX.ManufacturingPlace))

g.add((EX.hasIngredient, RDF.type, OWL.ObjectProperty))
g.add((EX.hasIngredient, RDFS.domain, EX.Product))
g.add((EX.hasIngredient, RDFS.range, EX.Ingredient))

g.add((EX.hasIngredientsText, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasIngredientsText, RDFS.domain, EX.Product))
g.add((EX.hasIngredientsText, RDFS.range, XSD.string))

# Ajouter les propriétés pour quantity et serving_size
g.add((EX.hasQuantity, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasQuantity, RDFS.domain, EX.Product))
g.add((EX.hasQuantity, RDFS.range, XSD.string))

#g.add((EX.hasServingSize, RDF.type, OWL.DatatypeProperty))
#g.add((EX.hasServingSize, RDFS.domain, EX.Product))
#g.add((EX.hasServingSize, RDFS.range, XSD.string))


g.add((EX.hasPurchasePlace, RDF.type, OWL.ObjectProperty))
g.add((EX.hasPurchasePlace, RDFS.domain, EX.Product))
g.add((EX.hasPurchasePlace, RDFS.range, EX.PurchasePlace))

g.add((EX.hasCity, RDF.type, OWL.ObjectProperty))
g.add((EX.hasCity, RDFS.domain, EX.Product))
g.add((EX.hasCity, RDFS.range, EX.City))

"""g.add((EX.hasEcoscore, RDF.type, OWL.ObjectProperty))
g.add((EX.hasEcoscore, RDFS.domain, EX.Product))
g.add((EX.hasEcoscore, RDFS.range, EX.Ecoscore))

g.add((EX.hasEcoscoreScore, RDF.type, OWL.ObjectProperty))
g.add((EX.hasEcoscoreScore, RDFS.domain, EX.Ecoscore))
g.add((EX.hasEcoscoreScore, RDFS.range, EX.EcoscoreScore))

#g.add((EX.hasEcoscoreGrade, RDF.type, OWL.ObjectProperty))
g.add((EX.hasEcoscoreGrade, RDFS.domain, EX.Ecoscore))
g.add((EX.hasEcoscoreGrade, RDFS.range, EX.EcoscoreGrade))"""


# Fonction pour encoder les URI de manière sécurisée
def encode_uri(uri):
    return urllib.parse.quote(uri, safe=":/#?&=~")


CACHE_FILE = "country_qcode_cache.json"

def load_cache():
    """
    Charge le cache des résultats depuis un fichier local.
    """
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_cache(cache):
    """
    Sauvegarde le cache des résultats dans un fichier local.
    """
    with open(CACHE_FILE, 'w') as f:
        json.dump(cache, f)

def normalize_country_name(country_name):
    """
    Nettoie le nom d'un pays pour le rendre standard.
    """
    return country_name.replace("en:", "").split(" - ")[0].strip()

def get_country_qcode(country_name, timeout=10):
    """
    Recherche le Q-code d'un pays donné via l'API SPARQL de Wikidata.
    Inclut des mécanismes de mise en cache et de gestion des erreurs.

    :param country_name: Nom du pays à rechercher
    :param timeout: Durée maximale pour la requête en secondes
    :return: URI complète du pays avec le Q-code, ou une valeur par défaut si une erreur survient
    """
    # Normaliser et nettoyer le nom du pays
    cleaned_country_name = normalize_country_name(country_name)

    # Charger le cache
    cache = load_cache()

    # Vérifier si le résultat est déjà dans le cache
    if cleaned_country_name in cache:
        print(f"Using cached result for {cleaned_country_name}")
        return cache[cleaned_country_name]

    # Configurer le point d'accès SPARQL
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

    # Construire la requête SPARQL
    query = f"""
    SELECT ?country WHERE {{
      ?country wdt:P31 wd:Q6256;  # Instance de "pays"
              rdfs:label "{cleaned_country_name}"@en .
    }}
    LIMIT 1
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Exécuter la requête
    try:
        sparql.setTimeout(timeout)  # Timeout pour éviter les blocages
        results = sparql.query().convert()
        for result in results.get("results", {}).get("bindings", []):
            qcode_uri = result["country"]["value"]
            # Ajouter le résultat au cache
            cache[cleaned_country_name] = qcode_uri
            save_cache(cache)
            return qcode_uri
    except requests.exceptions.RequestException as req_err:
        print(f"Request error for {country_name}: {req_err}")
    except Exception as e:
        print(f"Error during SPARQL query for {country_name}: {e}")

    # Retourner une valeur par défaut en cas d'erreur
    fallback_uri = "http://www.wikidata.org/entity/Q30"  # Par défaut : Q30 (USA)
    cache[cleaned_country_name] = fallback_uri
    save_cache(cache)
    return fallback_uri


def get_dbpedia_uri(brand_name):
    """Renvoie une URI DBpedia pour une marque donnée."""

    return f"http://dbpedia.org/resource/{brand_name.replace(' ', '_')}"


def get_wikidata_category_uri(category_name, language="en"):
    """
    Recherche le Q-code d'une catégorie donnée sur Wikidata.

    Args:
        category_name (str): Le nom de la catégorie à rechercher.
        language (str): La langue de recherche (par défaut "en" pour l'anglais).

    Returns:
        str: L'URI Wikidata pour la catégorie, ou None si elle n'est pas trouvée.
    """
    # URL de l'API Wikidata
    api_url = "https://www.wikidata.org/w/api.php"

    # Paramètres pour rechercher une entité correspondant à la catégorie
    params = {
        "action": "wbsearchentities",
        "search": category_name,
        "language": language,
        "format": "json",
        "type": "item"  # Recherche uniquement les items
    }

    try:
        # Requête à l'API Wikidata
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Lève une exception si le statut HTTP n'est pas 200

        # Récupérer les résultats de la recherche
        data = response.json()
        if data.get("search"):
            # Retourner le premier résultat (Q-code)
            q_code = data["search"][0]["id"]
            return f"http://www.wikidata.org/entity/{q_code}"
        else:

            return f"http://www.wikidata.org/entity/Q11070"
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la requête à l'API Wikidata : {e}")
        return None


# Lire le fichier CSV et ajouter des instances
csv_file_path = "cleaned_file.csv"  # Remplacez par votre fichier CSV

try:
    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        print(" debut lecture csv \n")
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logging.info("Début du traitement du fichier CSV.")
        for row in reader:
            if all(col in row for col in important_columns):
                product_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/{row['code']}"))

                # Ajouter le type de produit
                print(" Ajout produit \n")
                g.add((product_uri, RDF.type, DBO.Product))
                g.add((product_uri, SCHEMA.name, Literal(row["product_name"])))
                g.add((product_uri, SCHEMA.url, URIRef(row["url"])))

                # Ajouter la marque avec lien vers DBpedia
                print(" Ajout marque produit, association avec dbpedia \n")
                if row["brands"]:
                    brand_name = row["brands"].strip()
                    brand_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/brand/{row['brands']}"))
                    dbpedia_brand_uri = get_dbpedia_uri(row["brands"])
                    g.add((brand_uri, RDF.type, DBO.Brand))
                    g.add((brand_uri, FOAF.name, Literal(row["brands"])))
                    if dbpedia_brand_uri:

                        g.add((brand_uri, DBO.wikiPage, URIRef(encode_uri(dbpedia_brand_uri))))

                    g.add((product_uri, EX.hasBrand, brand_uri))


                # Ajouter des catégories basées sur le champ 'categories' avec lien vers Wikidata

                if row["categories"]:
                    categories = row["categories"].split(',')  # Séparer les catégories par des virgules
                    for category in categories:
                        category = category.strip()
                        if category:  # Vérifier que la catégorie n'est pas vide
                            category_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/category/{category}"))
                            wikidata_category_uri = get_wikidata_category_uri(category)
                            g.add((category_uri, RDF.type, EX.Category))
                            g.add((category_uri, SCHEMA.name, Literal(category)))
                            g.add((category_uri, DBO.wikiPage, URIRef(encode_uri(wikidata_category_uri))))

                            g.add((product_uri, EX.hasCategory, category_uri))


                # Ajouter des relations d'origine basées sur le champ 'countries' avec lien vers Wikidata

                if row["countries"]:
                    countries = row["countries"].split(',')  # Séparer les pays par des virgules
                    for country in countries:
                        country = country.strip()
                        if country:  # Vérifier que le pays n'est pas vide
                            country_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/country/{country}"))

                            normalized_country = normalize_country_name(country)

                            qcode_uri = get_country_qcode(normalized_country)

                            #wikidata_country_uri = get_wikidata_uri(country)

                            g.add((country_uri, RDF.type, DBO.Country))

                            g.add((country_uri, FOAF.name, Literal(normalized_country)))


                            g.add((country_uri, DBO.wikiPage, URIRef(encode_uri(qcode_uri))))

                            g.add((product_uri, EX.availableInCountry, country_uri))

                if row["allergens"]:
                    allergens = row["allergens"].split(',')  # Séparer les valeurs par des virgules
                    for allergen in allergens:
                        allergen = allergen.strip()
                        if allergen:  # Vérifier que l'origine n'est pas vide
                            # Créer une instance de l'origine
                            allergen_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/allergens/{allergen}"))

                            g.add((product_uri, EX.containsAllergen, allergen_uri))
                            g.add((allergen_uri, RDF.type, EX.Allergen))
                            g.add((allergen_uri, SCHEMA.name, Literal(allergen)))



                # Ajouter des nutriments
                nutrients = [
                    ("energy_100g", "Energy"),
                    ("fat_100g", "Fat"),
                    ("saturated-fat_100g", "Saturated Fat"),
                    ("carbohydrates_100g", "Carbohydrates"),
                    ("sugars_100g", "Sugars"),
                    ("fiber_100g", "Fiber"),
                    ("proteins_100g", "Proteins"),
                    ("salt_100g", "Salt"),
                    ("carbon-footprint_100g", "carbon footprint")
                ]

                for nutrient, label in nutrients:
                    if row[nutrient]:
                        nutrient_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/nutrient/{label}"))
                        g.add((product_uri, EX.hasNutrient, nutrient_uri))
                        g.add((nutrient_uri, RDF.type, EX.Nutrient))
                        g.add((nutrient_uri, SCHEMA.name, Literal(label)))
                        g.add((nutrient_uri, SCHEMA.value, Literal(row[nutrient], datatype=XSD.float)))


                # Ajouter des ingrédients

                if row["ingredients_text"]:
                    g.add((product_uri, EX.hasIngredientsText, Literal(row["ingredients_text"])))


                # Ajouter les créateurs

                if row["creator"]:
                    creator_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/creator/{row['creator']}"))
                    g.add((product_uri, EX.hasCreator, creator_uri))
                    g.add((creator_uri, RDF.type, EX.Creator))
                    g.add((creator_uri, SCHEMA.name, Literal(row["creator"])))


                # Ajouter les lieux de fabrication

                if row["manufacturing_places"]:
                    manufacturing = row["manufacturing_places"].split(',')  # Séparer les catégories par des virgules
                    for manu in manufacturing:
                        cmanu = manu.strip()
                        if category:  # Vérifier que la catégorie n'est pas vide
                            manufacturing_place_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/manufacturing_place/{manu}"))
                            g.add((product_uri, EX.hasManufacturingPlace, manufacturing_place_uri))
                            g.add((manufacturing_place_uri, RDF.type, EX.ManufacturingPlace))
                            g.add((manufacturing_place_uri, SCHEMA.name, Literal(manu)))

                if row["purchase_places"]:
                    purchase_place_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/purchase_place/{row['purchase_places']}"))
                    g.add((product_uri, EX.hasPurchasePlace, purchase_place_uri))
                    g.add((purchase_place_uri, RDF.type, EX.PurchasePlace))
                    g.add((purchase_place_uri, SCHEMA.name, Literal(row["purchase_places"])))

                # Ajouter les villes

                if row["cities"]:
                    city_uri = URIRef(encode_uri(f"http://produitsalimentaires.fr/city/{row['cities']}"))
                    g.add((product_uri, EX.hasCity, city_uri))
                    g.add((city_uri, RDF.type, EX.City))
                    g.add((city_uri, SCHEMA.name, Literal(row["cities"])))


                  # Ajouter la quantité (quantity)
                print(" Ajout quantite produit, association avec dbpedia \n")
                if row["quantity"]:
                    g.add((product_uri, EX.hasQuantity, Literal(row["quantity"])))


except Exception as e:
    print(f"Erreur lors du chargement du fichier CSV : {e}")
    exit()

# Sauvegarder le graphe OWL
g.serialize(destination=rdf_file_path, format="xml")
print(f"Ontologie OWL générée et sauvegardée dans : {rdf_file_path}")

 debut lecture csv 

Ontologie OWL générée et sauvegardée dans : output_ontology2.rdf
